# Iterate folders

In [27]:
import os
import glob
import cv2
import numpy as np
import pandas as pd

In [5]:
root = '/home/user/Dev/Git/roofcounting/'
# image_name = "pasirsari_1_labeled" 
# image_ext = ".png"
# image_ext = ".jpg"
# image_file = os.path.join("data/images/", (image_name + image_ext))
# image_path = os.path.join(root, image_file)
# image = cv2.imread(image_path)
# print(image_path)

# train = os.path.join(root, "data/train_data","images")
# test = os.path.join(root, "data/test_data", "images")

# path_sets = [train, test]

In [34]:
def pre_process(image):
    # adaptive threshold as preprocessing -> looks for areas that are relatively brighter than the neighbouring pixels
    blur_hor = cv2.filter2D(image[:, :, 0], cv2.CV_32F, kernel=np.ones((11,1,1), np.float32)/11.0, borderType=cv2.BORDER_CONSTANT)
    blur_vert = cv2.filter2D(image[:, :, 0], cv2.CV_32F, kernel=np.ones((1,11,1), np.float32)/11.0, borderType=cv2.BORDER_CONSTANT)
    mask = ((image[:,:,0]>blur_hor*1.2) | (image[:,:,0]>blur_vert*1.2)).astype(np.uint8)*255
    return mask

In [22]:
def hough_it_up(mask):
    circles = cv2.HoughCircles(mask,
                           cv2.HOUGH_GRADIENT,
                           minDist=2,
                           dp=1,
                           param1=150,
                           param2=12,
                           minRadius=4,
                           maxRadius=10)
    return circles

In [24]:
def to_csv(circles, csv_name):
    xy_array = []

    for i in range(len(circles[0])):
        xy = circles[0][i][0], circles[0][i][1]
        xy_array.append(xy)

    df = pd.DataFrame(xy_array, columns=["x", "y"])
    df.to_csv(csv_name, index=False)

In [19]:
label = os.path.join(root, "data/pts")        
labelled_paths = []
for path in glob.glob(os.path.join(label, "*.jpg")):
    labelled_paths.append(path)

In [36]:
for label_file in labelled_paths: 
#     print(label_file)
    image = cv2.imread(label_file)
    mask = pre_process(image)
    circles = hough_it_up(mask)
    to_csv(circles, label_file.replace("_pts", "").replace(".jpg", ".csv"))

## Move csv to train_data folder

In [45]:
os.system("""
    mv ../data/pts/*.csv ../data/train_data/images
""")

0